<a href="https://colab.research.google.com/github/thtuhin488/environment/blob/main/Andernach_ANN_with_time_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import sklearn as sklearn
import os

excel_file = '/sheet 1.xlsx'
# excel_file = 'Y:/Home/tuhin/New folder/sheet 1.xls'
Dt = pd.read_excel(excel_file,usecols=['Date'])
D = pd.read_excel(excel_file,usecols=['Discharge'])


In [12]:
dataset = D.values
dataset = dataset.astype('float32') 

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
dataset = scaler.fit_transform(dataset)

train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [15]:
def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(0, len(dataset) - seq_size - 1, 3):
        # print(i)
        window = dataset[i:(i + seq_size), 0]
        x.append(window)
        y.append(dataset[i + seq_size, 0])

    return np.array(x), np.array(y)


In [16]:

seq_size = 5 # Number of time steps to look back

# Larger sequences (look further back) may improve forecasting.
trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)

In [ ]:


print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))

#Input dimensions are... (N x seq_size)
print('Build deep model...')
# create and fit dense model
model = Sequential()
model.add(Dense(64, input_dim=seq_size, activation='relu'))
model.add(Dense(32 , activation='relu'))
model.add(Dense(32 , activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
print(model.summary())

#model.fit(trainX, trainY, epochs=200, batch_size=2, verbose=2)

model.fit(trainX, trainY, validation_data=(testX, testY),
          verbose=2, epochs=100)
# make predictions
trainPredict1 = model.predict(trainX)
testPredict1 = model.predict(testX)

predictions = model.predict(testX[:5])
print("Predicted values are: ", predictions)
print("Real values are: ", test[:5])

In [27]:

trainPredict = scaler.inverse_transform(trainPredict1)
trainY_inverse = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict1)
testY_inverse = scaler.inverse_transform([testY])


# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY_inverse[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY_inverse[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


#Neural network - from the current code
#mse_neural, mae_neural = model.evaluate(X_test_scaled, y_test)
mse_neural, mae_neural = model.evaluate(trainX,trainY)
print('Mean squared error from neural net for train: ', mse_neural)
print('Mean absolute error from neural net for train: ', mae_neural)

mse_neural, mae_neural = model.evaluate(testX,testY)
print('Mean squared error from neural net for test: ', mse_neural)
print('Mean absolute error from neural net for test: ', mae_neural)

print('R^2 Value from train: ', sklearn.metrics.r2_score(trainY_inverse[0], trainPredict[:,0]))
print('R^2 Value from test: ', sklearn.metrics.r2_score(testY_inverse[0], testPredict[:,0]))

###########



Train Score: 139.07 RMSE
Test Score: 118.85 RMSE
15/15 [==============================] - 0s 16ms/step - loss: 6.5018e-04 - acc: 0.0000e+00
Mean squared error from neural net for train:  0.0006501779425889254
Mean absolute error from neural net for train:  0.0
8/8 [==============================] - 0s 1ms/step - loss: 4.7483e-04 - acc: 0.0000e+00
Mean squared error from neural net for test:  0.00047483021626248956
Mean absolute error from neural net for test:  0.0
R^2 Value from train:  0.9768360087901147
R^2 Value from test:  0.984618121907175


In [28]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Linear regression
lr_model = linear_model.LinearRegression()
lr_model.fit(trainX,trainY)
y_pred_lr = lr_model.predict(testX)
mse_lr = mean_squared_error(testY, y_pred_lr)
mae_lr = mean_absolute_error(testY, y_pred_lr)
RMSE_lr = math.sqrt(mean_squared_error(testY, y_pred_lr))

print('test Score: %.2f RMSE' % (RMSE_lr))

print('Mean squared error from linear regression for test: ', mse_lr)
print('Mean absolute error from linear regression for test: ', mae_lr)

print('R^2 Value from test: ', sklearn.metrics.r2_score(testY, y_pred_lr))

lr_model.fit(trainX,trainY)
yt_pred_lr = lr_model.predict(trainX)
mse_lrt = mean_squared_error(trainY, yt_pred_lr)
mae_lrt = mean_absolute_error(trainY, yt_pred_lr)
RMSE_lrt = math.sqrt(mean_squared_error(trainY, yt_pred_lr))

print('train Score: %.2f RMSE' % (RMSE_lrt))
print('Mean squared error from linear regression for train: ', mse_lrt)
print('Mean absolute error from linear regression for train: ', mae_lrt)
print('R^2 Value from train: ', sklearn.metrics.r2_score(trainY, yt_pred_lr))


test Score: 0.02 RMSE
Mean squared error from linear regression for test:  0.00047081147
Mean absolute error from linear regression for test:  0.013740308
R^2 Value from test:  0.9847483082042556
train Score: 0.02 RMSE
Mean squared error from linear regression for train:  0.0005663733
Mean absolute error from linear regression for train:  0.01380639
R^2 Value from train:  0.9798217333693152


In [29]:
tree = DecisionTreeRegressor()
tree.fit(trainX, trainY)
y_pred_tree = tree.predict(testX)
mse_dt = mean_squared_error(testY, y_pred_tree)
mae_dt = mean_absolute_error(testY, y_pred_tree)
RMSE_dt = math.sqrt(mean_squared_error(testY, y_pred_tree))

print('R^2 Value from test using decision tree: ', sklearn.metrics.r2_score(testY,y_pred_tree ))
print('test Score: %.2f RMSEusing decisin tree' % (RMSE_dt))
print('Mean squared error using decision tree: ', mse_dt)
print('Mean absolute error using decision tree: ', mae_dt)


yt_pred_tree = tree.predict(trainX)
mse_dtt = mean_squared_error(trainY, yt_pred_tree)
mae_dtt = mean_absolute_error(trainY,yt_pred_tree)
RMSE_dtt = math.sqrt(mean_squared_error(trainY, yt_pred_tree))

print('train Score: %.2f RMSE' % (RMSE_dtt))
print('Mean squared error from decision tree for train: ', mse_dtt)
print('Mean absolute error from decision tree for train: ', mae_dtt)
print('R^2 Value from train using decision tree: ', sklearn.metrics.r2_score(trainY,  yt_pred_tree))

R^2 Value from test using decision tree:  0.9520401125782115
test Score: 0.04 RMSEusing decisin tree
Mean squared error using decision tree:  0.0014804958630020746
Mean absolute error using decision tree:  0.022799447782126517
train Score: 0.00 RMSE
Mean squared error from decision tree for train:  0.0
Mean absolute error from decision tree for train:  0.0
R^2 Value from train using decision tree:  1.0


In [30]:
#random forest 
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 30, random_state=30)
model.fit(trainX, trainY)

y_pred_RF = model.predict(testX)

mse_RF = mean_squared_error(testY, y_pred_RF)
mae_RF = mean_absolute_error(testY, y_pred_RF)
RMSE_RF = math.sqrt(mean_squared_error(testY, y_pred_RF))

print('R^2 Value from test using randomforest ', sklearn.metrics.r2_score(testY,y_pred_RF))
print('test Score: %.2f RMSE' % (RMSE_RF))
print('Mean squared error using Random Forest: ', mse_RF)
print('Mean absolute error Using Random Forest: ', mae_RF)

yt_pred_RF = model.predict(trainX)
mse_RFt = mean_squared_error(trainY, yt_pred_RF)
mae_RFt = mean_absolute_error(trainY,yt_pred_RF)
RMSE_RFt = math.sqrt(mean_squared_error(trainY, yt_pred_RF))

print('train Score: %.2f RMSE' % (RMSE_RFt))
print('Mean squared error from random forest for train: ', mse_RFt)
print('Mean absolute error from random forest for train: ', mae_RFt)
print('R^2 Value from train for random forest: ', sklearn.metrics.r2_score(trainY,  yt_pred_RF))

R^2 Value from test using randomforest  0.9718842389178356
test Score: 0.03 RMSE
Mean squared error using Random Forest:  0.0008679183835696093
Mean absolute error Using Random Forest:  0.018615095152465597
train Score: 0.01 RMSE
Mean squared error from random forest for train:  0.0001682195146335952
Mean absolute error from random forest for train:  0.00724316812120378
R^2 Value from train for random forest:  0.9940068170343284
